In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
printf <- function(...) cat(sprintf(...))

In [23]:
# valid columns when high speed
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'RoadType', 'flag', 'TimeToCollision', "RoadFactor")
isValidColumn <- function(x) ! x %in% invalids

In [24]:
CV <- function(dfx, expr, verbose=FALSE) {        
    folds <- createFolds(dfx$flag, k=5)
    
    count <- 1
    errs <- c()
    reds <- c()
    blues <- c()
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]
        # fit <- rpart(expr, data=train, method="class")
        fit <- rpart(expr, data=train, method="class", cp=0.022)
        p <- predict(fit, newdata=test)
        predictedFlags <- colnames(p)[max.col(p, ties.method = "first")]
        
        if (verbose) {
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, predictedFlags, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        } else {
            result <- correctVsPredict(test, predictedFlags, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
        }
        
        nerr <- sum((predictedFlags == test$flag) == FALSE)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(c(mean(errs), mean(reds), mean(blues)))
}

In [25]:
correctVsPredict <- function(test, predictedFlags, verbose=FALSE) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum((predictedRedRows$flag == 'Red') == TRUE)
    nPredictedReds <- nrow(predictedRedRows)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum((predictedBlueRows$flag == 'Blue') == TRUE)
    nPredictedBlues <- nrow(predictedBlueRows)
        
    if (verbose) {
        printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
        printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)  
    }    
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [26]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [27]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Reds

In [28]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [29]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [30]:
allFeatures <- c(colnames(df3))

In [31]:
roadTypes <- unique(df3$RoadType)

In [32]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    expr <- createFormula(invalids, allFeatures)
    result <- CV(dfx, expr, verbose=FALSE)
    printRedRatios(dfx)
    printf("Red: Mean correct/predict = %f\n", result[2])
    printf("Blue: Mean correct/predict = %f\n", result[3])
    printf("CV value: %f", result[1])
    printf("\n\n")
}

RoadType: 7 (その他)
Red/All = 25/27 = 0.925926
1 - Red/All = 2/27 = 0.074074
Red: Mean correct/predict = 0.933333
Blue: Mean correct/predict = NaN
CV value: 0.066667

RoadType: 6 (一般道1、一般道2、一般道3)
Red/All = 90/189 = 0.476190
1 - Red/All = 99/189 = 0.523810
Red: Mean correct/predict = 0.588895
Blue: Mean correct/predict = 0.611340
CV value: 0.407112

RoadType: 4 (県道)
Red/All = 36/71 = 0.507042
1 - Red/All = 35/71 = 0.492958
Red: Mean correct/predict = 0.513131
Blue: Mean correct/predict = 0.536667
CV value: 0.477143

RoadType: 5 (主要地方道)
Red/All = 154/303 = 0.508251
1 - Red/All = 149/303 = 0.491749
Red: Mean correct/predict = 0.598957
Blue: Mean correct/predict = 0.583743
CV value: 0.409617

RoadType: 3 (国道)
Red/All = 202/324 = 0.623457
1 - Red/All = 122/324 = 0.376543
Red: Mean correct/predict = 0.678337
Blue: Mean correct/predict = 0.528571
CV value: 0.364038

RoadType: 0 (都市間高速)
Red/All = 55/81 = 0.679012
1 - Red/All = 26/81 = 0.320988
Red: Mean correct/predict = 0.676190
Blue: Mean corr

# Divide More ...

In [15]:
df3 %>% group_by(RoadType) %>% summarize(counts=n())

RoadType,counts
0,81
2,47
3,324
4,71
5,303
6,189
7,27


# Threshold = 10, 30

In [16]:
set.seed(1)

In [17]:
# Not Use
dfx <- df3[df3$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [18]:
verbose = TRUE

In [19]:
i = 6
dfx <- df3[df3$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [20]:
i = 4
dfx <- df3[df3$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [21]:
i = 5
threshold1 <- 10
threshold2 <- 30
printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
dfx <- df3[df3$RoadType == i, ]
expr <- createFormula(invalids, allFeatures)
dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

result1 <- CV(dfx1, expr, verbose=verbose)        
printf("AverageVelocity <= %d:\n", threshold1)
printRedRatios(dfx1)        
printf("Red: Mean correct/predict = %f\n", result1[2])
printf("Blue: Mean correct/predict = %f\n", result1[3])
printf("CV value: %f", result1[1])        
printf("\n\n")
        
result2 <- CV(dfx2, expr, verbose=verbose)
printf("%d < AverageVelocity <= %d:\n", threshold1, threshold2)
printRedRatios(dfx2)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

result3 <- CV(dfx3, expr, verbose=verbose)        
printf("AverageVelocity > %d:\n", threshold2)
printRedRatios(dfx3)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

RoadType: 5 (主要地方道)
Fold1
As for Red: correct/predict = 3/5 = 0.600000
As for Blue: correct/predict = 5/9 = 0.555556

Fold2
As for Red: correct/predict = 4/9 = 0.444444
As for Blue: correct/predict = 3/6 = 0.500000

Fold3
As for Red: correct/predict = 4/8 = 0.500000
As for Blue: correct/predict = 3/5 = 0.600000

Fold4
As for Red: correct/predict = 4/6 = 0.666667
As for Blue: correct/predict = 5/8 = 0.625000

Fold5
As for Red: correct/predict = 4/6 = 0.666667
As for Blue: correct/predict = 5/7 = 0.714286

AverageVelocity <= 10:
Red/All = 33/69 = 0.478261
1 - Red/All = 36/69 = 0.521739
Red: Mean correct/predict = 0.575556
Blue: Mean correct/predict = 0.598968
CV value: 0.417656

Fold1
As for Red: correct/predict = 5/10 = 0.500000
As for Blue: correct/predict = 12/25 = 0.480000

Fold2
As for Red: correct/predict = 6/15 = 0.400000
As for Blue: correct/predict = 9/20 = 0.450000

Fold3
As for Red: correct/predict = 12/25 = 0.480000
As for Blue: correct/predict = 5/11 = 0.454545

Fold4
As for

In [22]:
i = 3
threshold1 <- 10
threshold2 <- 30
printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
dfx <- df3[df3$RoadType == i, ]
expr <- createFormula(invalids, allFeatures)
dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

result1 <- CV(dfx1, expr, verbose=verbose)        
printf("AverageVelocity <= %d:\n", threshold1)
printRedRatios(dfx1)        
printf("Red: Mean correct/predict = %f\n", result1[2])
printf("Blue: Mean correct/predict = %f\n", result1[3])
printf("CV value: %f", result1[1])        
printf("\n\n")
        
result2 <- CV(dfx2, expr, verbose=verbose)
printf("%d < AverageVelocity <= %d:\n", threshold1, threshold2)
printRedRatios(dfx2)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

result3 <- CV(dfx3, expr, verbose=verbose)        
printf("AverageVelocity > %d:\n", threshold2)
printRedRatios(dfx3)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

RoadType: 3 (国道)
Fold1
As for Red: correct/predict = 6/8 = 0.750000
As for Blue: correct/predict = 2/2 = 1.000000

Fold2
As for Red: correct/predict = 7/9 = 0.777778
As for Blue: correct/predict = 1/1 = 1.000000

Fold3
As for Red: correct/predict = 4/5 = 0.800000
As for Blue: correct/predict = 3/6 = 0.500000

Fold4
As for Red: correct/predict = 5/6 = 0.833333
As for Blue: correct/predict = 2/4 = 0.500000

Fold5
As for Red: correct/predict = 7/10 = 0.700000
As for Blue: correct/predict = 1/1 = 1.000000

AverageVelocity <= 10:
Red/All = 34/52 = 0.653846
1 - Red/All = 18/52 = 0.346154
Red: Mean correct/predict = 0.772222
Blue: Mean correct/predict = 0.800000
CV value: 0.267273

Fold1
As for Red: correct/predict = 19/27 = 0.703704
As for Blue: correct/predict = 4/5 = 0.800000

Fold2
As for Red: correct/predict = 17/26 = 0.653846
As for Blue: correct/predict = 3/7 = 0.428571

Fold3
As for Red: correct/predict = 12/17 = 0.705882
As for Blue: correct/predict = 7/16 = 0.437500

Fold4
As for Re